### Imports

In [1]:
from matplotlib import pyplot as plt
from data import load_data, load_narx_data
from matplotlib import pyplot as plt
import numpy as np
import torch
from data import load_narx_data
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, Matern, Product
from scipy.sparse import csr_matrix

In [2]:
n_a, n_b = 2, 2
Split = [0.6, 0.2, 0.2] # split; [training, validation, test]
total_number_of_points = 1000 # total number of points to consider from the larger dataset (starting from index 0)
restart = 2

In [3]:
Xtrain,Ytrain = load_narx_data(n_a, n_b, total_number_of_points, section="train", split=Split, as_tensor=True)
Xval,Yval = load_narx_data(n_a, n_b, total_number_of_points, section="validation", split=Split, as_tensor=True)
Xtest,Ytest = load_narx_data(n_a, n_b, total_number_of_points, section="test", split=Split, as_tensor=True)

Considering 1000 datapoints
Using 600 datapoints for training set
Considering 1000 datapoints
Using 200 datapoints for validation set
Considering 1000 datapoints
Using 200 datapoints for test set


In [4]:
# Standard settings
kernel = RBF(length_scale=1) + WhiteKernel(noise_level=0.1)
reg = GaussianProcessRegressor(kernel, n_restarts_optimizer=restart)

Create sparse matrices

In [5]:
XtrainSparse = csr_matrix(Xtrain)
YtrainSparse = csr_matrix(Ytrain)

In [6]:
Yval = Yval.numpy()  # Convert Yval from Tensor to NumPy array
Ytest = Ytest.numpy()  # Convert Ytest from Tensor to NumPy array

## Fit the GP

In [7]:
reg.fit(XtrainSparse.toarray(), Ytrain)

GaussianProcessRegressor(kernel=RBF(length_scale=1) + WhiteKernel(noise_level=0.1),
                         n_restarts_optimizer=2)

In [8]:
F = lambda u,y: reg.predict(np.concatenate([u,y])[None,:])[0] 

try:
    yval_sim = load_narx_data(2,2,Xval, F)
    error_val = np.mean((yval_sim-Yval)**2)**0.5/np.std(Yval)
except:
    print("Problems detected")

Problems detected


In [13]:
def get_NARX_data(ulist, data, na, nb):
    # iteratively uses the given f to find the new output.
    
    # init upast and ypast as lists.
    upast = [0]*nb
    ypast = [0]*na
    
    ylist = []
    for index,unow in enumerate(ulist):
        #compute the current y given by f
        print(unow)
        ynow = data[index]
        
        #update past arrays
        upast.append(unow) #add an element on the end
        upast.pop(0)       #remove the fist element
        ypast.append(ynow)
        ypast.pop(0)
        
        #save result
        ylist.append(ynow)
    return np.array(ylist) #return result

In [14]:
model_now = reg
Xtest, Ytest = load_narx_data(n_a, n_b, total_number_of_points, section="test", split=Split, as_tensor=True)
Ytest_pred = model_now.predict(Xtest)
Ytest = Ytest.numpy()  # Convert Ytest from Tensor to NumPy array
print('NRMS test prediction:', np.mean((Ytest-Ytest_pred)**2)**0.5/np.std(Ytest))

# try:
ytest_sim = get_NARX_data(Xtest,Ytest, n_a, n_b) #explodes
NRMS_sim = np.mean((Ytest-ytest_sim)**2)**0.5/np.std(Ytest)
plt.plot(Ytest)
plt.plot(Ytest-ytest_sim)
plt.show()
# except ValueError:
#     NRMS_sim = float('inf')
print('NRMS test simulation:', NRMS_sim)

Considering 1000 datapoints
Using 200 datapoints for test set
NRMS test prediction: 0.009406777310260567
tensor([-1.9307, -1.1234, -0.3110, -0.2293], dtype=torch.float64)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
Yval_pred, Yval_pred_std = reg.predict(Xval, return_std=True) 

NRMS = np.mean((Yval_pred-Yval)**2)**0.5/np.std(Yval)

print(f'Validation NRMS= {NRMS}')

In [ ]:
print(reg.kernel_)

Make some nice plots

In [ ]:
# Plotting
Ytrain_pred, Ytrain_pred_std = reg.predict(Xtrain, return_std=True) 
plt.figure(figsize=(12, 5)) 
plt.plot(Ytrain) 
plt.title('Prediction on the training set')
plt.errorbar(np.arange(len(Xtrain)), Ytrain_pred, yerr=2 * Ytrain_pred_std, fmt='.r') 
plt.grid()
plt.xlabel('Sample')
plt.ylabel('y')
plt.legend(['Measured', 'Pred'])
plt.xlim([0, 250])
plt.show()

plt.figure(figsize=(12, 5)) 
plt.title('Prediction on the validation set')
plt.plot(Yval) 
Yval_pred, Yval_pred_std = reg.predict(Xval, return_std=True) 
plt.errorbar(np.arange(len(Xval)), Yval_pred, yerr=2 * Yval_pred_std, fmt='.r') 
plt.grid()
plt.xlabel('Sample')
plt.ylabel('y')
plt.legend(['Measured', 'Pred']) 
plt.xlim([0, 250])
plt.show()


print(f'Validation NRMS= {np.mean((Yval_pred-Yval)**2)**0.5/np.std(Yval)}')

plt.figure(figsize=(12, 5))
plt.title('Prediction on the test set')
plt.plot(Ytest)
ytest_pred, ytest_pred_std = reg.predict(Xtest, return_std=True)
plt.errorbar(np.arange(len(Xtest)), ytest_pred, yerr=2 * ytest_pred_std, fmt='.r')
plt.grid()
plt.xlabel('Sample')
plt.ylabel('y')
plt.legend(['Measured', 'Pred'])
plt.xlim([0, 250])
plt.show()

print(f'Test NRMS = {np.mean((ytest_pred - Ytest) ** 2, axis=0) ** 0.5 / np.std(Ytest)}')

# Results

Time calculation: <br>
1000 items takes 0.7 sec